In [89]:
import data_fun as data
import json
import pandas as pd
import random
import re

# Province and City Chinese-English map

In [315]:
CNEG_df = pd.read_csv('china_regions/provinceCN_EN.csv', encoding="gb2312")  

In [316]:
CNEG_df.head()

,City,City_Admaster,City_EN,Province,Province_EN,Region,Tier
0,七台河,七台河市,Qitaihe,黑龙江,Heilongjiang,North,Tier 5
1,三亚,三亚市,Sanya,海南,Hainan,South,Tier 3
2,三明,三明市,Sanming,福建,Fujian,East,Tier 3
3,三门峡,三门峡市,Sanmenxia,河南,Henan,North,Tier 5
4,上海,上海市,Shanghai,上海,Shanghai,East,Tier 1


In [317]:
ProvinceCNENMap = pd.Series(CNEG_df.Province_EN.values, index=CNEG_df.Province).to_dict()

In [318]:
ProvinceCNENMap['台湾省'] = 'Taiwan'
ProvinceCNENMap['香港特别行政区'] = 'Xianggang'
ProvinceCNENMap['澳门特别行政区'] = 'Aomen'

In [319]:
ProvinceCNENMap['内蒙古自治区'] = ProvinceCNENMap['内蒙古']
del ProvinceCNENMap['内蒙古']

ProvinceCNENMap['广西壮族自治区'] = ProvinceCNENMap['广西']
del ProvinceCNENMap['广西']

ProvinceCNENMap['西藏自治区'] = ProvinceCNENMap['西藏']
del ProvinceCNENMap['西藏']

ProvinceCNENMap['宁夏回族自治区'] = ProvinceCNENMap['宁夏']
del ProvinceCNENMap['宁夏']

ProvinceCNENMap['新疆维吾尔自治区'] = ProvinceCNENMap['新疆']
del ProvinceCNENMap['新疆']

In [320]:
for key, value in ProvinceCNENMap.items():
    if not key.endswith('省') and not key.endswith('行政区') and not key.endswith('自治区'):
        newKey = key + '省'
        ProvinceCNENMap[newKey] = ProvinceCNENMap[key]
        del ProvinceCNENMap[key]

In [321]:
ProvinceCNENMap['江西省'] = ProvinceCNENMap['江西']
del ProvinceCNENMap['江西']

ProvinceCNENMap['广东省'] = ProvinceCNENMap['广东']
del ProvinceCNENMap['广东']

ProvinceCNENMap['北京市'] = ProvinceCNENMap['北京省']
del ProvinceCNENMap['北京省']

ProvinceCNENMap['天津市'] = ProvinceCNENMap['天津省']
del ProvinceCNENMap['天津省']

ProvinceCNENMap['重庆市'] = ProvinceCNENMap['重庆省']
del ProvinceCNENMap['重庆省']

ProvinceCNENMap['上海市'] = ProvinceCNENMap['上海']
del ProvinceCNENMap['上海']

In [322]:
ProvinceCNENMap

{'台湾省': 'Taiwan',
 '香港特别行政区': 'Xianggang',
 '澳门特别行政区': 'Aomen',
 '内蒙古自治区': 'Neimenggu',
 '广西壮族自治区': 'Guangxi',
 '西藏自治区': 'Xizang',
 '宁夏回族自治区': 'Ningxia',
 '新疆维吾尔自治区': 'Xinjiang',
 '黑龙江省': 'Heilongjiang',
 '海南省': 'Hainan',
 '福建省': 'Fujian',
 '河南省': 'Henan',
 '山东省': 'Shandong',
 '山西省': 'Shanxi',
 '云南省': 'Yunnan',
 '辽宁省': 'Liaoning',
 '浙江省': 'Zhejiang',
 '四川省': 'Sichuan',
 '安徽省': 'Anhui',
 '河北省': 'Hebei',
 '贵州省': 'Guizhou',
 '甘肃省': 'Gansu',
 '湖北省': 'Hubei',
 '江苏省': 'Jiangsu',
 '吉林省': 'Jilin',
 '陕西省': 'Shanxi',
 '湖南省': 'Hunan',
 '青海省': 'Qinghai',
 '江西省': 'Jiangxi',
 '广东省': 'Guangdong',
 '北京市': 'Beijing',
 '天津市': 'Tianjin',
 '重庆市': 'Chongqing',
 '上海市': 'Shanghai'}

In [324]:
CityCNENMap = pd.Series(CNEG_df.City_EN.values, index=CNEG_df.City_Admaster).to_dict()

# Construct Province-City-County maps

In [598]:
ProvinceDf = pd.read_csv('china_regions/province.csv', encoding="utf8")

In [599]:
ProvinceDf

,code,name,province
0,110000,北京市,11
1,120000,天津市,12
2,130000,河北省,13
3,140000,山西省,14
4,150000,内蒙古自治区,15
5,210000,辽宁省,21
6,220000,吉林省,22
7,230000,黑龙江省,23
8,310000,上海市,31
9,320000,江苏省,32


In [600]:
provinceList = ProvinceDf['name'].tolist()

In [601]:
len(provinceList)

34

In [626]:
cityDf = pd.read_csv('china_regions/city.csv', encoding="utf8")

In [627]:
cityDf

,code,name,province,city
0,130100,石家庄市,13,1
1,130200,唐山市,13,2
2,130300,秦皇岛市,13,3
3,130400,邯郸市,13,4
4,130500,邢台市,13,5
...,...,...,...,...
330,654000,伊犁哈萨克自治州,65,40
331,654200,塔城地区,65,42
332,654300,阿勒泰地区,65,43
333,999999,诸暨市,33,99


In [628]:
cityList = cityDf['name'].tolist()

In [629]:
len(cityList)

335

In [630]:
countyDf = pd.read_csv('china_regions/area.csv', encoding="utf8")

In [607]:
# ---***--- IMPORTANT
countyList = countyDf['name'].tolist()

In [608]:
# get the index of a county in the countyList
countyList.index('吉木萨尔县')

2763

In [164]:
# get city name based on province and city ID from the city dataframe
cityDf.loc[cityDf['city']==1].loc[cityDf['province']==13]['name'].values[0]

'石家庄市'

In [166]:
# get province name based on province ID from the province dataframe
ProvinceDf.loc[ProvinceDf['province'] == 13]['name'].values[0]

'河北省'

In [183]:
# get province based on city name
provinceID = cityDf.loc[cityDf['name']=='阳江市']['province'].values[0]
ProvinceDf.loc[ProvinceDf['province'] == provinceID]['name'].values[0]

'广东省'

In [174]:
# get province ID and city ID from county dataframe at index 0
proID = countyDf.iloc[[0]]['province'].values[0]
cityID = countyDf.iloc[[0]]['city'].values[0]

# Read and Parse data functions

In [697]:
def parseFactLocale(fact):
    
    zhiXiaShi = ['天津市', '北京市','天津市','上海市','重庆市']
    
    shengCN = 'None'
    shengEN = 'None'
    shiCN = 'None'
    shiEN = 'None'
    quXianCN = 'None'
    
    # (1) when "...人民检察院指控" is included. Split string use that
    segments = fact.split('人民检察院指控') # '人民检察院'
    if len(segments) > 1:
        PATTERN = u'([\u4e00-\u9fa5]{2,5}?(?:省|自治区|市)){0,1}([\u4e00-\u9fa5]{2,7}?(?:市|区|县|州)){0,1}([\u4e00-\u9fa5]{2,7}?(?:市|区|县)){0,1}'

        data_utf8 = segments[0]
    
        province = ''
        city = ''
        district = ''
        
        chunk1 = None
        chunk2 = None
        chunk3 = None

        pattern = re.compile(PATTERN)
        m = pattern.search(data_utf8)
        
        if len(m.group(0)) == 0:
#             print('bad entry')
            return shengCN, shengEN, shiCN, shiEN, quXianCN
        
        if m.lastindex >= 1:
            chunk1 = m.group(1)
        if m.lastindex >= 2:
            chunk2 = m.group(2)
        if m.lastindex >= 3:
            chunk3 = m.group(3)
        
        if chunk1:
            if chunk1 in zhiXiaShi:
                shengCN = chunk1
                shiCN = chunk1
                
            elif chunk1.endswith('省') or chunk1.endswith('自治区'):
                shengCN = chunk1
                if chunk2:
                    if chunk2.endswith('市'):
                        shiCN = chunk2
                        if chunk3:
                            quXianCN = chunk3
                    else: #chunk2 ends with 区|县
                        quXianCN = chunk2
                        try:
                            countyIdx = countyList.index(chunk2)
                            proID = countyDf.iloc[[countyIdx]]['province'].values[0]
                            cityID = countyDf.iloc[[countyIdx]]['city'].values[0]
                            shiCN = cityDf.loc[cityDf['city']==cityID].loc[cityDf['province']==proID]['name'].values[0]
                        except:
                            nothing='no function. just holding place'
#                             print('cannot find: ' + chunk2)
                        
            elif chunk1.endswith('市') and chunk1 != '四会市' and chunk1 != '湘乡市':
                shiCN = chunk1
                try: 
                    provinceID = cityDf.loc[cityDf['name']== chunk1]['province'].values[0]
                    shengCN = ProvinceDf.loc[ProvinceDf['province'] == provinceID]['name'].values[0]
                except:
#                     print('市: ' + chunk1 + ' is not in the 市级file')
                    try:
                        countyIdx = countyList.index(chunk1)
                        proID = countyDf.iloc[[countyIdx]]['province'].values[0]
                        shengCN = ProvinceDf.loc[ProvinceDf['province'] == proID]['name'].values[0]
                        cityID = countyDf.iloc[[countyIdx]]['city'].values[0]
                        shiCN = cityDf.loc[cityDf['city']==cityID].loc[cityDf['province']==proID]['name'].values[0]
#                         print('found ' + chunk1 + 'in the 县级 file')
                    except:
                        nothing='no function. just holding place'
#                         print('cannot find: ' + chunk1)
                
                
                    
            else: #chunk1 endswith 区|县
                quXianCN = chunk1
                try:
                    countyIdx = countyList.index(chunk1)
                    proID = countyDf.iloc[[countyIdx]]['province'].values[0]
                    shengCN = ProvinceDf.loc[ProvinceDf['province'] == proID]['name'].values[0]
                    cityID = countyDf.iloc[[countyIdx]]['city'].values[0]
                    shiCN = cityDf.loc[cityDf['city']==cityID].loc[cityDf['province']==proID]['name'].values[0]
                except:
                    nothing='no function. just holding place'
#                     print('cannot find: ' + chunk1)
                
    
    else: # cannot split on “人民检察院指控"
        for province in provinceList:
            if province in fact:
                shengCN = province
                break
        
        for city in cityList:
            if city in fact:
                shiCN = city
                provinceID = cityDf.loc[cityDf['name']== shiCN]['province'].values[0]
                shengCN = ProvinceDf.loc[ProvinceDf['province'] == provinceID]['name'].values[0]
                break
        
        for county in countyList:
            if county in fact:
                quXianCN = county
                try:
                    countyIdx = countyList.index(county)
                    proID = countyDf.iloc[[countyIdx]]['province'].values[0]
                    shengCN = ProvinceDf.loc[ProvinceDf['province'] == proID]['name'].values[0]
                    if shengCN in zhiXiaShi:
                        shiCN = shengCN
                    else:
                        cityID = countyDf.iloc[[countyIdx]]['city'].values[0]
                        shiCN = cityDf.loc[cityDf['city']==cityID].loc[cityDf['province']==proID]['name'].values[0]
                except:
                    nothing='no function. just holding place'
#                     print('error at: ' + county)
                    
                break
    
    if shengCN != 'None':
        try:
            shengEN = ProvinceCNENMap[shengCN]
        except:
            nothing='no function. just holding place'
#             print('省: ' + shengCN + 'does not exist')
            shengCN = 'None'
        
    if shiCN != 'None':
        if shiCN in CityCNENMap:
            shiEN = CityCNENMap[shiCN]
    
    return shengCN, shengEN, shiCN, shiEN, quXianCN

In [612]:
ranInt = random.randint(0, 1710856)
testFact = alltext[ranInt]
testFact

'泉州市鲤城区人民检察院指控，2015年6月6日0时许，被告人徐某某在鲤城区金龙街道鸿星尔克鞋厂附近其亲戚杨某的租房内吃饭喝酒后，驾驶二轮摩托车离开，沿南福南路、南环路往南安方向行驶，行至南环路超越集团红绿灯路段被公安机关当场查获。经抽血检验，被告人徐某某血液中乙醇浓度达132.59mg／100ml，属醉酒驾驶。\r\n'

In [613]:
shengCN, shengEN, shiCN, shiEN, quXianCN  = parseFactLocale(testFact)
print('shengCN: ' + shengCN)
print('shengEN: ' + shengEN)
print('shiCN: ' + shiCN)
print('shiEN: ' + shiEN)
print('quXianCN: ' + quXianCN)

shengCN: 福建省
shengEN: Fujian
shiCN: 泉州市
shiEN: Quanzhou
quXianCN: None


In [725]:
def parseFactTime(factData):
    
    mat = re.search(r"(\d{4}年(?:\d{1,2}月){0,1}(?:\d{1,2}日){0,1})", factData)
    #print(mat.groups())
    # ('2016-12-12',)
    
    year = '0000'
    month = '01'
    date = '01'
    
    if mat:
        dateString = mat.groups()[0]
        yearSplit = dateString.split('年')
        year = yearSplit[0]
        if yearSplit[1]:
            monthSplit = yearSplit[1].split('月')
            month = monthSplit[0]
            if monthSplit[1]:
                date = monthSplit[1].split('日')[0]
    
    return year + '-' + month + '-' + date


In [726]:
mat = re.search(r"(\d{4}年\d{1,2}月(?:\d{1,2}日){0,1})",'永顺县人民检察院指控，2014年1月11日，被告人李某某与彭某某（另案处理）在永顺县塔卧镇')
print(mat.groups()[0])

2014年1月11日


In [727]:
ok = parseFactTime('乌鲁木齐县人民检察院指控：2014年，被告人魏某在担任乌')
ok

'2014-01-01'

In [713]:
def parseData(path):
    fin = open(path, 'r', encoding = 'utf8')
    
    province_CN = []
    province_EN = []
    city_CN = []
    city_EN = []
    County_CN = []
    crimeTime_earliest = []
    
    facttext = []
    lawArticle = []
    accusation = []
    moneyPunishment = []
    criminalName = []
    deathPenalty = []
    imprisonment = []
    lifeImprisonment = []

    line = fin.readline()

    while line:
        d = json.loads(line)
        
        factData = d['fact'].rstrip()
        
        facttext.append(factData)
        lawArticle.append(d['meta']['relevant_articles'][0])
        accusation.append(d['meta']['accusation'][0]) 
        moneyPunishment.append(d['meta']['punish_of_money'])
        criminalName.append(d['meta']['criminals'][0])
        deathPenalty.append(d['meta']['term_of_imprisonment']['death_penalty'])
        imprisonment.append(d['meta']['term_of_imprisonment']['imprisonment'])
        lifeImprisonment.append(d['meta']['term_of_imprisonment']['life_imprisonment'])
        
        shengCN, shengEN, shiCN, shiEN, quXianCN = parseFactLocale(factData)
        dateTime = parseFactTime(factData)
        province_CN.append(shengCN)
        province_EN.append(shengEN)
        city_CN.append(shiCN)
        city_EN.append(shiEN)
        County_CN.append(quXianCN)
        crimeTime_earliest.append(dateTime)
        
        line = fin.readline()
    fin.close()
    
    #load into DataFrame
    df = pd.DataFrame(list(zip(facttext, lawArticle, accusation, moneyPunishment, criminalName, deathPenalty,
                              imprisonment, lifeImprisonment, crimeTime_earliest, province_CN, province_EN,
                              city_CN, city_EN, County_CN)), 
                      columns =['fact', 'lawArticle', 'accusation', 'moneyPunishment', 'criminalName', 'deathPenalty',
                               'imprisonment', 'lifeImprisonment', 'dateTime', 'province_CN', 'province_EN',
                               'city_CN', 'city_EN', 'County_CN'])

    df.to_csv('./dataframes/firstStage_train_updatedDatetime.csv', index=False)
    return facttext, accusation, lawArticle, imprisonment

In [714]:
alltext, accu_label, law_label, imprisonment = parseData('../CAIL2018_final_all_data/first_stage/train.json')

In [699]:
len(alltext)

1710856